# Desafio Escale

Este <i>notebook</i> e um estudo passo a passo do dataset fornecido. Será utilizada tecnologia Spark em Python para a resolução do desafio.

No primeiro momento sera instalado e importadas bibliotecas do Spark

In [1]:
try:
    !pip install pyspark=="2.4.5" --quiet
    !pip install pandas=="1.0.4" --quiet
except:
    print("Running throw py file.")

In [134]:
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark import SparkFiles
from pyspark.sql.types import StringType, FloatType
import pyspark
import json
import pandas as pd
import numpy as np

Criação de uma sessão Spark

In [3]:
spark = SparkSession\
        .builder\
        .appName("Desafio Data Engineer Escale - Fabio Kfouri")\
        .getOrCreate()
spark

Para otimizar a resolução, foram realizados downloads dos datasets, esta lógica é para identificar se este notebook esta rodando máquina do autor, caso positivo, utilizará o dataset local, do contrário, utilizará o dataset da núvem.

In [4]:
import os

dataPath = 'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/'

if 'E:\\' in os.getcwd() and 'dataEngineerTest_Escale' in os.getcwd():
    dataPath = os.getcwd() + "/data/"

print(dataPath)


E:\Projetos\Jobs\dataEngineerTest_Escale/data/


## Leitura do primeiro dataset
Os códigos seguintes tem o objetivo apenas de conhecer o dataset:
- vizualizar alguns registros
- quantidade de registros
- observar o Schema do datase

In [5]:
df0 = spark.read.json(dataPath + 'part-0000' + str(0) +'.json.gz')
df0.show(3)

+--------------------+--------------+----------------+---------------------+--------+---+---------+--------+-------+
|        anonymous_id|browser_family|   device_family|device_sent_timestamp|   event|  n|os_family|platform|version|
+--------------------+--------------+----------------+---------------------+--------+---+---------+--------+-------+
|46074eab-28f7-483...| Chrome Mobile|Samsung SM-A105M|        1592616733778|pageview| 82|  Android|     web|    1.0|
|24c1f50c-c192-4f8...| Chrome Mobile|Samsung SM-A105M|        1592615846912|pageview| 82|  Android|     web|    1.0|
|2c7ed100-3e30-4c1...| Chrome Mobile|Samsung SM-A105M|        1592624340522|pageview| 82|  Android|     web|    1.0|
+--------------------+--------------+----------------+---------------------+--------+---+---------+--------+-------+
only showing top 3 rows



In [6]:
df0.count()

10235120

In [7]:
df0.printSchema()

root
 |-- anonymous_id: string (nullable = true)
 |-- browser_family: string (nullable = true)
 |-- device_family: string (nullable = true)
 |-- device_sent_timestamp: long (nullable = true)
 |-- event: string (nullable = true)
 |-- n: long (nullable = true)
 |-- os_family: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- version: string (nullable = true)



## Construção do ClickStream

O código seguinte é para identificar os usuários que mais utilizaram o site. Esta pesquisa é importante para poder criar o <b>Sessionamento</b>.

In [8]:
df = df0.groupBy('anonymous_id').count().sort(F.col("count").desc())
df.show(10,False)

+------------------------------------+-----+
|anonymous_id                        |count|
+------------------------------------+-----+
|ae59b395-31ac-40e3-b7c0-3d97d00ac6cc|293  |
|7d0f437b-23e1-407e-8a08-6fb4434bc1f7|3    |
|e0ad55f0-29de-4c01-8da1-d37eb2f3ad4c|3    |
|9bf50c34-836a-4f03-b9da-bd90d84c8cd3|2    |
|68f73aec-f9c3-4d39-9cba-0454a4972bdd|2    |
|34d49ea1-7c32-4d6c-a3df-40455e509ec7|2    |
|b08777e1-3bd8-4cfc-9415-a830b0764c77|2    |
|b70140df-d9c2-49eb-9eb1-6ae9ab77fb48|2    |
|88222d44-f5da-4710-98ef-89c39b635a9d|2    |
|880d31a8-8181-4e33-b65c-99f26a448f3e|2    |
+------------------------------------+-----+
only showing top 10 rows



Identificado um <b>anonymous_id</b> com uso intensivo do site, que será utilizado para a modelagem do sessionamento.

In [9]:
df = df0.filter('anonymous_id = "ae59b395-31ac-40e3-b7c0-3d97d00ac6cc"').sort(F.col("device_sent_timestamp"))
df.show()

+--------------------+--------------+-------------+---------------------+--------+---+---------+--------+-------+
|        anonymous_id|browser_family|device_family|device_sent_timestamp|   event|  n|os_family|platform|version|
+--------------------+--------------+-------------+---------------------+--------+---+---------+--------+-------+
|ae59b395-31ac-40e...|     Googlebot|       Spider|        1591321597456|pageview| 95|    Other|     web|    1.0|
|ae59b395-31ac-40e...|     Googlebot|       Spider|        1591414551868|pageview| 69|    Other|     web|    1.0|
|ae59b395-31ac-40e...|     Googlebot|       Spider|        1591673040878|pageview| 73|    Other|     web|    1.0|
|ae59b395-31ac-40e...| AdsBot-Google|       Spider|        1591733995529|pageview| 64|    Other|     web|    1.0|
|ae59b395-31ac-40e...|     Googlebot|       Spider|        1591756611249|pageview| 43|    Other|     web|    1.0|
|ae59b395-31ac-40e...|     Googlebot|       Spider|        1592014689622|pageview| 45|  

Para calculo do tempo de sessão será utilizado a função <b>LAG()</b> que tem por objetivo trazer no registro corrente o dado de timestamp do registro anterior.

O autor assumiu como premissa não declarada para definir uma sessão, que além do tempo limite de 30 minutos desde a última utilização, que uma sessão precisaria considerar o <b>device_family</b> e <b>os_family</b>.

Ou seja, mesmo que não tenha excedido o tempo limite de 30 minutos, mas se for caracterizado que houve uma mudança de device_family ou os_family, trata-se de uma sessão nova.

In [110]:
overCategory = Window.partitionBy('anonymous_id','device_family','os_family','browser_family')\
                    .orderBy('anonymous_id','device_family','os_family','browser_family','device_sent_timestamp')

In [111]:
dftemp = df.withColumn("lag", F.lag('device_sent_timestamp', 1).over(overCategory))
dftemp.show(5)

+--------------------+--------------+-------------+---------------------+--------+---+---------+--------+-------+-------------+
|        anonymous_id|browser_family|device_family|device_sent_timestamp|   event|  n|os_family|platform|version|          lag|
+--------------------+--------------+-------------+---------------------+--------+---+---------+--------+-------+-------------+
|ae59b395-31ac-40e...|     Googlebot|       Spider|        1592102553331|pageview| 55|  Android|     web|    1.0|         null|
|ae59b395-31ac-40e...|     Googlebot|       Spider|        1592617192847|pageview| 78|  Android|     web|    1.0|1592102553331|
|ae59b395-31ac-40e...|     Googlebot|       Spider|        1592708051282|pageview| 44|  Android|     web|    1.0|1592617192847|
|ae59b395-31ac-40e...|     Googlebot|       Spider|        1592708655174|pageview| 50|  Android|     web|    1.0|1592708051282|
|ae59b395-31ac-40e...|     Googlebot|       Spider|        1592714671326|pageview| 96|  Android|     web

Identificado que o tempo esta no formato <b>epoch</b>. 

A primeira intuição do autor foi de converter para o formato de data, porém, logo o autor percebeu que poderia usar o própria valor numérico para identificar a tolerancia de 30 minutos.

Para isso, foi convertido o tempo de 30 minutos em milisegundos para o uso simplificado na identificação da sessão.

In [112]:
time_limit = (30*60)*1000
print(time_limit)

1800000


In [113]:
dftemp = dftemp.withColumn('delta', F.col('device_sent_timestamp') - F.col('lag'))\
        .withColumn('same_section', (F.col('device_sent_timestamp') - F.col('lag')) < time_limit) \
        .withColumn('event_time', (F.col('device_sent_timestamp')/1000).cast('timestamp'))

dftemp.select('device_family','os_family','browser_family','device_sent_timestamp','same_section','event_time')\
        .show(20, False)

+-------------+---------+--------------+---------------------+------------+-----------------------+
|device_family|os_family|browser_family|device_sent_timestamp|same_section|event_time             |
+-------------+---------+--------------+---------------------+------------+-----------------------+
|Spider       |Android  |Googlebot     |1592102553331        |null        |2020-06-13 23:42:33.331|
|Spider       |Android  |Googlebot     |1592617192847        |false       |2020-06-19 22:39:52.847|
|Spider       |Android  |Googlebot     |1592708051282        |false       |2020-06-20 23:54:11.282|
|Spider       |Android  |Googlebot     |1592708655174        |true        |2020-06-21 00:04:15.174|
|Spider       |Android  |Googlebot     |1592714671326        |false       |2020-06-21 01:44:31.326|
|Spider       |Android  |Googlebot     |1592787487408        |false       |2020-06-21 21:58:07.408|
|Spider       |Android  |Googlebot     |1592790085073        |false       |2020-06-21 22:41:25.073|


Devido a demora de processamento na maquina do autor, optou-se por gerar um dataset menor para analise.

In [114]:
dftemp.coalesce(1).write.format("json").save("analise.json")

In [115]:
dftemp = spark.read.json(os.getcwd() + '/analise.json')

Criacão de uma view chamada clickStream.

In [116]:
dftemp.createOrReplaceTempView("raw_table")

Pimeiro passo foi possível observar as sessoes abertas e fazer um especie de <i>smoke test</i>.

In [117]:
df_sessioned = spark.sql("""

      SELECT anonymous_id, browser_family, delta, device_family, device_sent_timestamp, event, event_time, n, os_family, --
             platform, nvl(same_section,false) same_section, version,
             case when nvl(same_section,false) = false then
                'session_' || ROW_NUMBER() OVER (PARTITION BY anonymous_id,nvl(same_section,false) ORDER BY device_sent_timestamp )
             else
                null
             end
             as partial_session
        FROM raw_table t --
        order by anonymous_id, device_sent_timestamp 
""")
df_sessioned.select('device_family','device_sent_timestamp','delta','same_section','event_time','partial_session').show(30, False)

+-------------+---------------------+---------+------------+-----------------------------+---------------+
|device_family|device_sent_timestamp|delta    |same_section|event_time                   |partial_session|
+-------------+---------------------+---------+------------+-----------------------------+---------------+
|Spider       |1591321597456        |null     |false       |2020-06-04T22:46:37.456-03:00|session_1      |
|Spider       |1591414551868        |92954412 |false       |2020-06-06T00:35:51.868-03:00|session_2      |
|Spider       |1591673040878        |258489010|false       |2020-06-09T00:24:00.878-03:00|session_3      |
|Spider       |1591733995529        |null     |false       |2020-06-09T17:19:55.529-03:00|session_4      |
|Spider       |1591756611249        |83570371 |false       |2020-06-09T23:36:51.249-03:00|session_5      |
|Spider       |1592014689622        |258078373|false       |2020-06-12T23:18:09.622-03:00|session_6      |
|Spider       |1592102553331        |

In [118]:
df_sessioned = spark.sql("""
with temp as (--
      SELECT anonymous_id, browser_family, delta, device_family, device_sent_timestamp, event, event_time, n, os_family, --
             platform, nvl(same_section,false) same_section, version,
             case when nvl(same_section,false) = false then
             'session_' || ROW_NUMBER() OVER (PARTITION BY anonymous_id,nvl(same_section,false) ORDER BY device_sent_timestamp )
             else
              null
             end
             as partial_session
        FROM raw_table t --
), table_section_id as (--
    select anonymous_id, browser_family, delta, device_family, device_sent_timestamp, event, event_time, n, os_family, --
             platform, nvl(same_section,false) same_section, version,
             LAST_VALUE(partial_session,True) OVER (PARTITION BY anonymous_id ORDER BY device_sent_timestamp ) || '_' || anonymous_id session_id
    from temp t
)
select * from table_section_id
order by anonymous_id, device_sent_timestamp 
""")
df_sessioned.show()

+--------------------+--------------+---------+-------------+---------------------+--------+--------------------+---+---------+--------+------------+-------+--------------------+
|        anonymous_id|browser_family|    delta|device_family|device_sent_timestamp|   event|          event_time|  n|os_family|platform|same_section|version|          session_id|
+--------------------+--------------+---------+-------------+---------------------+--------+--------------------+---+---------+--------+------------+-------+--------------------+
|ae59b395-31ac-40e...|     Googlebot|     null|       Spider|        1591321597456|pageview|2020-06-04T22:46:...| 95|    Other|     web|       false|    1.0|session_1_ae59b39...|
|ae59b395-31ac-40e...|     Googlebot| 92954412|       Spider|        1591414551868|pageview|2020-06-06T00:35:...| 69|    Other|     web|       false|    1.0|session_2_ae59b39...|
|ae59b395-31ac-40e...|     Googlebot|258489010|       Spider|        1591673040878|pageview|2020-06-09T00

Neste ponto temos efetivamente um ClickStream

In [119]:
df_sessioned.createOrReplaceTempView("clickstream")

## Desafio 1

Calcular a quantidade total de sessões únicas por arquivo do conjunto de dados e apresentar no formato JSON.

Neste passo, foi realizado o filtro para desconsiderar as sessoes abertas para descobrir o total de sessoes únicas.

In [120]:
df_question_1 = spark.sql("""
select * from clickstream
where same_section = false
order by anonymous_id, device_sent_timestamp 
  
""")
df_question_1.select('device_family','device_sent_timestamp','same_section','event_time','session_id').show(30, False)

+-------------+---------------------+------------+-----------------------------+-----------------------------------------------+
|device_family|device_sent_timestamp|same_section|event_time                   |session_id                                     |
+-------------+---------------------+------------+-----------------------------+-----------------------------------------------+
|Spider       |1591321597456        |false       |2020-06-04T22:46:37.456-03:00|session_1_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc |
|Spider       |1591414551868        |false       |2020-06-06T00:35:51.868-03:00|session_2_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc |
|Spider       |1591673040878        |false       |2020-06-09T00:24:00.878-03:00|session_3_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc |
|Spider       |1591733995529        |false       |2020-06-09T17:19:55.529-03:00|session_4_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc |
|Spider       |1591756611249        |false       |2020-06-09T23:36:51.249-03:00|session_5_ae59b39

No terceiro e passo foi calculada a quantidade de sessoes abertas, neste caso, para um único usuário.

In [121]:
df_question_1 = spark.sql("""
SELECT anonymous_id, COUNT(session_id)  qtd_session
FROM clickstream
where same_section = false
group by anonymous_id
""")
df_question_1.show()

+--------------------+-----------+
|        anonymous_id|qtd_session|
+--------------------+-----------+
|ae59b395-31ac-40e...|         71|
+--------------------+-----------+



# Desafio 2
Calcular a quantidade de sessões únicas que ocorreram em cada Browser, Sistema Operacional e Dispositivo dentro de todo o conjunto de dados.

No primeiro passo foi identificar da quantidade por <b>browser_family</b>, <b>os_family</b> e <b>device_family</b>.

In [122]:
spark.sql("""

SELECT browser_family, COUNT(session_id)  qtd_session
FROM clickstream
where same_section = false
group by browser_family --

""").show()

+--------------+-----------+
|browser_family|qtd_session|
+--------------+-----------+
| AdsBot-Google|          1|
|     Googlebot|         70|
+--------------+-----------+



In [123]:
spark.sql("""
SELECT os_family, COUNT(session_id)  qtd_session
FROM clickstream
where same_section = false
group by os_family
""").show()

+---------+-----------+
|os_family|qtd_session|
+---------+-----------+
|    Other|         39|
|  Android|         32|
+---------+-----------+



In [124]:
spark.sql("""
SELECT 'device_family' what, device_family ref, COUNT(session_id)  qtd_session
FROM clickstream
where same_section = false
group by device_family
""").show()

+-------------+------+-----------+
|         what|   ref|qtd_session|
+-------------+------+-----------+
|device_family|Spider|         71|
+-------------+------+-----------+



No segundo passo foi agrupar todos esses resultados em um dataframe

In [125]:
df_question_2 = spark.sql("""
with table_temp as (--
    SELECT *
    FROM clickstream
    where same_section = false
), 
table_union as (--
    SELECT 'device_family' what, device_family ref, COUNT(session_id)  qtd_session
      FROM table_temp
     group by device_family
    union
    SELECT 'os_family' what, os_family ref, COUNT(session_id)  qtd_session
      FROM table_temp
    group by os_family
    union
    SELECT 'browser_family' what, browser_family ref, COUNT(session_id)  qtd_session
      FROM table_temp
     group by browser_family --
)
select * from table_union
order by what, ref
""")
df_question_2.show()

+--------------+-------------+-----------+
|          what|          ref|qtd_session|
+--------------+-------------+-----------+
|browser_family|AdsBot-Google|          1|
|browser_family|    Googlebot|         70|
| device_family|       Spider|         71|
|     os_family|      Android|         32|
|     os_family|        Other|         39|
+--------------+-------------+-----------+



No passo seguinte foi de concatenar a coluna 'ref' com 'qtd_session' em um formato json e agrupar por collecao.

In [126]:
d2 = df_question_2.orderBy('what','ref')\
        .withColumn('item', F.concat(F.lit('"'), 'ref',F.lit('":'),'qtd_session'))\
        .groupBy('what').agg(F.array_join(F.collect_list('item'), delimiter=',').alias('collection'))
d2.show(10, False)

+--------------+--------------------------------+
|what          |collection                      |
+--------------+--------------------------------+
|device_family |"Spider":71                     |
|os_family     |"Android":32,"Other":39         |
|browser_family|"AdsBot-Google":1,"Googlebot":70|
+--------------+--------------------------------+



Finalmente construir a saida Json do desafio

In [127]:
question2 = {}
for row in d2.collect():
    items = str(row["collection"]).replace('"','').split(',')
    obj = {}
    for item in items:
        element = str(item).split(':')
        obj[element[0]] = int(element[1])       
    
    question2[row["what"]] = obj

print(question2)

{'device_family': {'Spider': 71}, 'os_family': {'Android': 32, 'Other': 39}, 'browser_family': {'AdsBot-Google': 1, 'Googlebot': 70}}


In [128]:
print(json.dumps(question2))

{"device_family": {"Spider": 71}, "os_family": {"Android": 32, "Other": 39}, "browser_family": {"AdsBot-Google": 1, "Googlebot": 70}}


## Desafio 3

Calcular a mediana da duração (em segundos) entre todas sessões únicas para cada segmento.

In [129]:
spark.sql("""

SELECT browser_family, COUNT(session_id)  qtd_session, avg(delta) media
FROM clickstream
group by browser_family --

""").show()

+--------------+-----------+--------------------+
|browser_family|qtd_session|               media|
+--------------+-----------+--------------------+
| AdsBot-Google|          1|                null|
|     Googlebot|        292|1.2832178227586206E7|
+--------------+-----------+--------------------+



In [139]:
ddf = spark.sql("""

SELECT device_family,os_family,browser_family, delta/1000 delta_seg, event_time  , same_section, session_id                 
FROM clickstream
where session_id = 'session_31_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc'

""")
ddf.show(300, False)

+-------------+---------+--------------+---------+-----------------------------+------------+-----------------------------------------------+
|device_family|os_family|browser_family|delta_seg|event_time                   |same_section|session_id                                     |
+-------------+---------+--------------+---------+-----------------------------+------------+-----------------------------------------------+
|Spider       |Android  |Googlebot     |2823.347 |2020-06-23T23:33:39.322-03:00|false       |session_31_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc|
|Spider       |Other    |Googlebot     |55.781   |2020-06-23T23:34:20.142-03:00|true        |session_31_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc|
|Spider       |Other    |Googlebot     |73.586   |2020-06-23T23:35:33.728-03:00|true        |session_31_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc|
|Spider       |Other    |Googlebot     |53.704   |2020-06-23T23:36:27.432-03:00|true        |session_31_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc|
|Spide

In [135]:
#https://stackoverflow.com/questions/38743476/how-to-find-the-median-in-apache-spark-with-python-dataframe-api/38743477
def find_median(values_list):
    try:
        median = np.median(values_list) #get the median of values in a list in each row
        return round(float(median),2)
    except Exception:
        return None #if there is anything wrong with the given values

median_finder = F.udf(find_median,FloatType())

In [142]:
ddfa = ddf.groupBy('session_id').agg(F.collect_list('delta_seg').alias('deltas'))


In [143]:
ddfa.show()

+--------------------+--------------------+
|          session_id|              deltas|
+--------------------+--------------------+
|session_31_ae59b3...|[2823.347, 55.781...|
+--------------------+--------------------+



In [66]:
df_question_1 = spark.sql("""
with temp as (--
      SELECT anonymous_id, browser_family, delta, device_family, device_sent_timestamp, event, event_time, n, os_family, --
             platform, nvl(same_section,false) same_section, version,
             case when nvl(same_section,false) = false then
             'session_' || ROW_NUMBER() OVER (PARTITION BY anonymous_id,nvl(same_section,false) ORDER BY device_sent_timestamp )
             else
              null
             end
             as partial_session
        FROM clickstream t --
), table_section_id as (--
    select anonymous_id, browser_family, delta, device_family, device_sent_timestamp, event, event_time, n, os_family, --
             platform, nvl(same_section,false) same_section, version,
             LAST_VALUE(partial_session,True) OVER (PARTITION BY anonymous_id ORDER BY device_sent_timestamp ) || '_' || anonymous_id session_id
    from temp t
)
select * from table_section_id
order by anonymous_id, device_sent_timestamp 
""")
df_question_1.select('device_family','os_family','browser_family','same_section','event_time','session_id').show(300, False)

+-------------+---------+--------------+------------+-----------------------------+-----------------------------------------------+
|device_family|os_family|browser_family|same_section|event_time                   |session_id                                     |
+-------------+---------+--------------+------------+-----------------------------+-----------------------------------------------+
|Spider       |Other    |Googlebot     |false       |2020-06-04T22:46:37.456-03:00|session_1_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc |
|Spider       |Other    |Googlebot     |false       |2020-06-06T00:35:51.868-03:00|session_2_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc |
|Spider       |Other    |Googlebot     |false       |2020-06-09T00:24:00.878-03:00|session_3_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc |
|Spider       |Other    |AdsBot-Google |false       |2020-06-09T17:19:55.529-03:00|session_4_ae59b395-31ac-40e3-b7c0-3d97d00ac6cc |
|Spider       |Other    |Googlebot     |false       |2020-06-09T23:36:51.249

In [54]:
df_question_1 = spark.sql("""
with temp as (--
      SELECT anonymous_id, browser_family, delta, device_family, device_sent_timestamp, event, event_time, n, os_family, --
             platform, nvl(same_section,false) same_section, version,
             case when nvl(same_section,false) = false then
             'session_' || ROW_NUMBER() OVER (PARTITION BY anonymous_id,nvl(same_section,false) ORDER BY device_sent_timestamp )
             else
              null
             end
             as partial_session
        FROM clickstream t --
), table_section as (--
    select anonymous_id, browser_family, delta, device_family, device_sent_timestamp, event, event_time, n, os_family, --
             platform, nvl(same_section,false) same_section, version,
             LAST_VALUE(partial_session,True) OVER (PARTITION BY anonymous_id ORDER BY device_sent_timestamp ) session_id
    from temp t
)
select session_id, count(1)
from table_section
group by session_id
order by session_id
""")
df_question_1.show(300, False)

+----------+--------+
|session_id|count(1)|
+----------+--------+
|session_1 |1       |
|session_10|1       |
|session_11|1       |
|session_12|1       |
|session_13|2       |
|session_14|2       |
|session_15|1       |
|session_16|1       |
|session_17|1       |
|session_18|16      |
|session_19|10      |
|session_2 |1       |
|session_20|1       |
|session_21|1       |
|session_22|2       |
|session_23|1       |
|session_24|1       |
|session_25|18      |
|session_26|19      |
|session_27|1       |
|session_28|3       |
|session_29|5       |
|session_3 |1       |
|session_30|11      |
|session_31|25      |
|session_32|8       |
|session_33|1       |
|session_34|15      |
|session_35|13      |
|session_36|2       |
|session_37|11      |
|session_38|15      |
|session_39|10      |
|session_4 |1       |
|session_40|1       |
|session_41|1       |
|session_42|2       |
|session_43|1       |
|session_44|3       |
|session_45|10      |
|session_46|5       |
|session_47|8       |
|session_4

In [ ]:
broadcastVar = sc.broadcast(df_question_1.collectAsMap(), 0)

In [ ]:
accum = sc.accumulator(spark,0)


In [ ]:
accum.value

In [ ]:
def sss(s):
    if (s!=True):
        return 'session_' # + str(accum.value)
    return None # + str(accum.value)

In [ ]:
session_udf = F.udf(lambda x: 
                    'session_' + str(accum.value) if s!=True
                    else x
                    ,
                     StringType())

In [ ]:
session_udf = F.udf(lambda x: sss(x),
                     StringType())

In [ ]:
#overCategory
dftemp.select('device_family','device_sent_timestamp','same_section','event_time')\
    .withColumn('session_id', session_udf('same_section'))\
    .withColumn('session_id', F.last('session_id', True).over(overCategory))\
    .show(30, False)

In [ ]:
#overCategory
dftemp.withColumn('session_id', session_udf('same_section'))\
    .withColumn('session_id', F.last('session_id', True).over(overCategory))\
    .show(30, False)

In [32]:
dftemp.coalesce(1).write.format("csv").save("analise.scv")